In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import sys
#hyper-parameters
INPUT_LAYER_SIZE=784
OUTPUT_LAYER_SIZE=10
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [21]:
train=pd.read_csv('./data/train.csv')
val=pd.read_csv('./data/val.csv')
test=pd.read_csv('./data/test.csv')

In [32]:
training_data_class =train.iloc[:,-1].copy().as_matrix()
training_data=train.iloc[:,1:785].copy().as_matrix()/255.0

In [33]:
validating_data_class =val.iloc[:,-1].copy().as_matrix()
validating_data=val.iloc[:,1:785].copy().as_matrix()/255.0

In [34]:
training_data=sk.preprocessing.scale(training_data)
validating_data=sk.preprocessing.scale(validating_data)

In [43]:
print (training_data[:,5])

[-0.07072376  0.10359759 -0.07072376 ..., -0.07072376 -0.07072376
  0.10359759]


In [31]:
clf = PCA()
clf.fit(training_data)


PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

In [35]:
training_data=clf.transform(training_data)
validating_data=clf.transform(validating_data)

In [36]:
clf.explained_variance_

array([  1.73134033e+02,   1.13011717e+02,   4.28177404e+01,
         3.98982013e+01,   3.17925857e+01,   2.36743809e+01,
         2.15403962e+01,   1.81383573e+01,   1.32716181e+01,
         1.03380522e+01,   9.17312793e+00,   7.54038085e+00,
         7.09979826e+00,   6.75254574e+00,   5.82810864e+00,
         5.72642095e+00,   5.18747712e+00,   4.94912109e+00,
         4.87570814e+00,   4.57574130e+00,   4.08695381e+00,
         4.00712655e+00,   3.71130474e+00,   3.52971531e+00,
         3.43293017e+00,   3.26916452e+00,   3.10913693e+00,
         3.05622595e+00,   2.96132772e+00,   2.92934129e+00,
         2.84470704e+00,   2.75753953e+00,   2.63336552e+00,
         2.59141039e+00,   2.56605603e+00,   2.48640302e+00,
         2.38995488e+00,   2.31950199e+00,   2.22245692e+00,
         2.21176931e+00,   2.15983171e+00,   2.10193235e+00,
         2.05659424e+00,   1.99395109e+00,   1.93734733e+00,
         1.88029232e+00,   1.86487461e+00,   1.78897071e+00,
         1.74046395e+00,

In [29]:
training_data=sk.preprocessing.normalize(training_data)
validating_data=sk.preprocessing.normalize(validating_data)

In [30]:
training_data=np.vstack((training_data,validating_data))
#training_data_class=np.concatenate((training_data_class,validating_data_class))
training_data.shape

(60000, 784)

In [6]:
def create_output(class_labels):
    '''returns one-hot vectors'''
    output=np.zeros((len(class_labels),10))
    for i in range(len(class_labels)):
        output[i,class_labels[i]]=1
    return output
def sigmoid(value):
    return 1/(1+np.exp(-value))
def tanh(value):
    return np.tanh(value)
def relu(value):
    return [max(0,i) for i in value]
def relu_dife(value):
    if value>0:
        return 1
    return 0
def relu_dif(value):
    return [relu_dife(i) for i in value]
def sigmoid_dif(value):
    return sigmoid(value)*(1-sigmoid(value))
def tanh_dif(value):
    return 1-tanh(value)*tanh(value)
def softmax(vector):
    vector=np.array(vector,dtype=np.float64)
    if sum(vector)!=sum(vector):
        #print vector
        sys.exit()
    num= np.exp(vector)
    
    return num/np.sum(num)
def paraCopy(w,b,multiplier=1):
    return [i*multiplier for i in w],[i*multiplier for i in b]
def paraAdd(A,B):
    a,b=A
    c,d=B
    return [i+j for i,j in zip(a,c)],[i+j for i,j in zip(b,d)]
def cliper(A):
    return A/np.linalg.norm(A)
#def cliper(A,B):
#    return [i/np.linalg.norm(i) for i in A],[j/np.linalg.norm(j) for j in B]

In [7]:
def shaper(lst):
    for i in lst:
        print 'shape -',i.shape
def loss(list1,list2,loss_type):
    if loss_type=='ce': #ce 
        return -np.sum([j*np.log2(i) for i,j in zip(list1,list2)])
    if loss_type=='sq': #sq
        return 0.5*np.sum([(i-j)**2 for i,j in zip(list1,list2)])
def dumpModel(model):
    kp=0
    for i,j in zip(model.weights,model.biases):
        np.save('./temp/weights_'+str(kp),i)
        np.save('./temp/biases_'+str(kp),j)
        kp+=1


In [8]:
class NN_Model:
    def __init__(self,lr=0.01,momentum=0,hidden_layers=(100,),activation='sigmoid',loss='sq',
                 opt='gd',batch_size=1,anneal=False,save_dir='./temp/',expt_dir='./temp/',max_iter=50):
        self.lr=lr
        self.momentum=momentum
        self.layers=(INPUT_LAYER_SIZE,)+hidden_layers+(OUTPUT_LAYER_SIZE,)
        if activation=='sigmoid':
            self.activation=sigmoid
            self.activation_diff=sigmoid_dif
        elif activation=='tanh':
            self.activation=tanh
            self.activation_diff=tanh_dif
        elif activation=='relu':
            self.activation=relu
            self.activation_diff=relu_dif
        else:
            print 'Error : activation function not found'
        self.activation_name=activation        
        
        self.loss=loss
        self.opt=opt
        self.batch_size=batch_size
        self.anneal=anneal
        self.save_dir=save_dir
        self.expt_dir=expt_dir
        self.num_layers=len(self.layers)
        self.max_iter=max_iter
    
    def __forward_propagation(self):
        h_set=[]
        a_set=[]
        h=self.input_data[TRAINER]
        h_set.append(h) #experimental
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            
            h=self.activation(a)
            if self.activation_name=='relu':
                h=cliper(h)
            a_set.append(a)
            h_set.append(h)
        a=self.biases[L]+np.matmul(self.weights[L],h)
        a_set.append(a)
        #print a
        y=softmax(a)
        #sys.exit()
        #h_set.append(y) #experimental
        #print h.shape
            
        return h_set,a_set,y
    
    def __forward_propagation_test(self,h):
        L=self.num_layers-2 # 2 are input and output layers
        for k in range(L): 
            a=self.biases[k]+np.matmul(self.weights[k],h)
            h=self.activation(a)
            if self.activation_name=='relu':
                h=cliper(h)
            
        a=self.biases[L]+np.matmul(self.weights[L],h)
        y=softmax(a)
        #h_set.append(y) #experimental
        #print h.shape
            
        return y
    
    
    def __back_propagation(self,h,a,y):
        L=self.num_layers-2
        dWeights=[]
        dBiases=[]
        if self.loss=='ce':
            daL_loss=-(self.output_data[TRAINER]-y) # for cross-entropy loss function
        elif self.loss=='sq':
            daL_loss=np.array([2*sum([(y[i]-self.output_data[TRAINER][i])*y[i]*((i==j)*1-y[j]) for i in range(len(y))]) for j in range(len(y))])
        else :
            print 'Wrong loss function'
            return
        dA_loss=daL_loss
        for k in range(L,-1,-1):
            #print 'k=',k
            dW_loss=np.outer(dA_loss,h[k]).T
            dB_loss=dA_loss
            
            if k!=0:
                dH_loss=np.matmul(self.weights[k].T,dA_loss)
            
                dA_loss=dH_loss*self.activation_diff(a[k-1])
            
            
            dWeights=[dW_loss.T]+dWeights
            dBiases=[dB_loss.T]+dBiases
            #print dA_loss.shape
            #return _,_
            
            
        return dWeights,dBiases
    
    def __update(self,dWeights,dBiases):
        for i in range(len(self.weights)):
            self.weights[i]=self.weights[i]-dWeights[i]
            self.biases[i]=self.biases[i]-dBiases[i]

    
    
    
    def __train(self):
        global TRAINER
        for i in range(self.max_iter):
            if i%5==0 and i!=0 and self.anneal==True:
                self.lr/=2
            
            accuracy=0
            if self.batch_size==1:
                self.batch_size=len(training_data)
            
            sets=len(training_data)/self.batch_size # number of batches
                
            oldDWeights,oldDBiases=paraCopy(self.weights,self.biases,0) # for momentum
            for j in range(sets):
                #print 'SET=',j
                los=0
                dWeights,dBiases=paraCopy(self.weights,self.biases,0)
                for TRAINER in range(j*self.batch_size,(j+1)*self.batch_size):
                    #print 'TRAINER=',TRAINER
                    h,a,y=self.__forward_propagation()

                    
                    dWeights,dBiases=paraAdd(self.__back_propagation(h,a,y),
                                         paraCopy(dWeights,dBiases,1))
                    
                    los+=loss(y,self.output_data[TRAINER],self.loss)

                dWeights,dBiases=paraAdd(paraCopy(oldDWeights,oldDBiases,self.momentum),
                                         paraCopy(dWeights,dBiases,self.lr))

                self.__update(dWeights,dBiases)


                
                oldDWeights,oldDBiases=paraCopy(dWeights,dBiases,1)

                #print np.argmax(y),self.raw_class_labels[TRAINER]
                #if np.argmax(y)==self.raw_class_labels[TRAINER]:

                 #   accuracy+=1
                print 'Epoch : ',i,'Step : ',j,'loss : ',los
                #print 'acc=',accuracy*1.0/sets
            #print 'validation loss ',sum([loss(i,)])

    def fit(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.random.uniform(low=-1.0,high=1.0,size=(self.layers[i+1],self.layers[i])))
            self.biases.append(np.random.uniform(low=-1.0,high=1.0,size=(self.layers[i+1])))
            
            #print self.biases[-1].shape
        self.__train()
        
    def resume(self,X,Y):
        self.input_data=X
        self.output_data=create_output(Y)
        self.weights=[]
        self.biases=[]
        self.raw_class_labels=Y
        for i in range(self.num_layers-1):
            self.weights.append(np.load('./temp/weights_'+str(i)+'.npy'))
            self.biases.append(np.load('./temp/biases_'+str(i)+'.npy'))
            #print self.weights[-1].shape
        self.__train()
        
        
    def predict(self,X):
        return [np.argmax(self.__forward_propagation_test(i)) for i in X]    
        
    

In [18]:
model=NN_Model(hidden_layers=(100,),lr=0.0001,max_iter=20,momentum=0.1,batch_size=25,anneal=True,loss='ce',
               activation='relu')
training_data=np.array(training_data)

In [19]:
model.fit(training_data,training_data_class)
#model.resume()

Epoch :  0 Step :  0 loss :  101.202887333
Epoch :  0 Step :  1 loss :  102.763443311
Epoch :  0 Step :  2 loss :  99.2036514133
Epoch :  0 Step :  3 loss :  82.7309758931
Epoch :  0 Step :  4 loss :  89.8548268076
Epoch :  0 Step :  5 loss :  80.8447848017
Epoch :  0 Step :  6 loss :  85.2123407893
Epoch :  0 Step :  7 loss :  95.4614360585
Epoch :  0 Step :  8 loss :  95.4502445238
Epoch :  0 Step :  9 loss :  105.451309848
Epoch :  0 Step :  10 loss :  103.305581608
Epoch :  0 Step :  11 loss :  76.2144177289
Epoch :  0 Step :  12 loss :  100.771335467
Epoch :  0 Step :  13 loss :  88.4817571215
Epoch :  0 Step :  14 loss :  82.2128228939
Epoch :  0 Step :  15 loss :  101.005549629
Epoch :  0 Step :  16 loss :  86.9710485834
Epoch :  0 Step :  17 loss :  83.7276354018
Epoch :  0 Step :  18 loss :  87.354455538
Epoch :  0 Step :  19 loss :  87.5545528329
Epoch :  0 Step :  20 loss :  81.3027549598
Epoch :  0 Step :  21 loss :  83.9588813105
Epoch :  0 Step :  22 loss :  83.3410105912

Epoch :  0 Step :  187 loss :  64.787432771
Epoch :  0 Step :  188 loss :  65.2678667447
Epoch :  0 Step :  189 loss :  66.0489895065
Epoch :  0 Step :  190 loss :  57.1020955832
Epoch :  0 Step :  191 loss :  66.9158678448
Epoch :  0 Step :  192 loss :  68.8094604182
Epoch :  0 Step :  193 loss :  58.3995203071
Epoch :  0 Step :  194 loss :  59.766415052
Epoch :  0 Step :  195 loss :  62.2335194146
Epoch :  0 Step :  196 loss :  55.8175198286
Epoch :  0 Step :  197 loss :  56.1492567971
Epoch :  0 Step :  198 loss :  64.406774917
Epoch :  0 Step :  199 loss :  59.8632835429
Epoch :  0 Step :  200 loss :  62.3346570327
Epoch :  0 Step :  201 loss :  70.5384199558
Epoch :  0 Step :  202 loss :  58.2372273213
Epoch :  0 Step :  203 loss :  67.4847829685
Epoch :  0 Step :  204 loss :  66.5068185719
Epoch :  0 Step :  205 loss :  71.0263865752
Epoch :  0 Step :  206 loss :  62.2120840132
Epoch :  0 Step :  207 loss :  59.4790981939
Epoch :  0 Step :  208 loss :  60.5181919644
Epoch :  0 St

Epoch :  0 Step :  373 loss :  47.2465190239
Epoch :  0 Step :  374 loss :  66.3409085984
Epoch :  0 Step :  375 loss :  48.1942082717
Epoch :  0 Step :  376 loss :  54.2666513149
Epoch :  0 Step :  377 loss :  53.427366143
Epoch :  0 Step :  378 loss :  53.716770305
Epoch :  0 Step :  379 loss :  59.744821694
Epoch :  0 Step :  380 loss :  54.6171073153
Epoch :  0 Step :  381 loss :  52.9544586952
Epoch :  0 Step :  382 loss :  58.311224126
Epoch :  0 Step :  383 loss :  53.061935819
Epoch :  0 Step :  384 loss :  58.3767131236
Epoch :  0 Step :  385 loss :  55.8799109239
Epoch :  0 Step :  386 loss :  52.6220370518
Epoch :  0 Step :  387 loss :  46.4162192911
Epoch :  0 Step :  388 loss :  55.5175289175
Epoch :  0 Step :  389 loss :  48.5653820685
Epoch :  0 Step :  390 loss :  61.2659643327
Epoch :  0 Step :  391 loss :  48.69993824
Epoch :  0 Step :  392 loss :  55.128883877
Epoch :  0 Step :  393 loss :  59.6865481249
Epoch :  0 Step :  394 loss :  56.249151355
Epoch :  0 Step :  

Epoch :  0 Step :  557 loss :  43.7993154062
Epoch :  0 Step :  558 loss :  62.8379169607
Epoch :  0 Step :  559 loss :  44.3268225769
Epoch :  0 Step :  560 loss :  51.4200559202
Epoch :  0 Step :  561 loss :  47.7636315138
Epoch :  0 Step :  562 loss :  51.7340329723
Epoch :  0 Step :  563 loss :  47.9701313874
Epoch :  0 Step :  564 loss :  48.4997874968
Epoch :  0 Step :  565 loss :  51.8133616167
Epoch :  0 Step :  566 loss :  57.3849007157
Epoch :  0 Step :  567 loss :  50.5116549097
Epoch :  0 Step :  568 loss :  47.2143182269
Epoch :  0 Step :  569 loss :  42.0345126877
Epoch :  0 Step :  570 loss :  47.591190792
Epoch :  0 Step :  571 loss :  43.521944706
Epoch :  0 Step :  572 loss :  55.9663581678
Epoch :  0 Step :  573 loss :  50.2603828779
Epoch :  0 Step :  574 loss :  35.25792526
Epoch :  0 Step :  575 loss :  45.9064523379
Epoch :  0 Step :  576 loss :  44.6292266004
Epoch :  0 Step :  577 loss :  48.1127231469
Epoch :  0 Step :  578 loss :  50.9172127057
Epoch :  0 Ste

Epoch :  0 Step :  741 loss :  44.9547958984
Epoch :  0 Step :  742 loss :  46.8394193277
Epoch :  0 Step :  743 loss :  47.4778423784
Epoch :  0 Step :  744 loss :  41.3245238215
Epoch :  0 Step :  745 loss :  46.4830465062
Epoch :  0 Step :  746 loss :  40.5560309219
Epoch :  0 Step :  747 loss :  57.6607474052
Epoch :  0 Step :  748 loss :  39.9066543676
Epoch :  0 Step :  749 loss :  42.6739310749
Epoch :  0 Step :  750 loss :  40.5344256346
Epoch :  0 Step :  751 loss :  42.6534991739
Epoch :  0 Step :  752 loss :  45.9231327926
Epoch :  0 Step :  753 loss :  46.0340414675
Epoch :  0 Step :  754 loss :  53.859884345
Epoch :  0 Step :  755 loss :  46.7076967081
Epoch :  0 Step :  756 loss :  50.1719512725
Epoch :  0 Step :  757 loss :  46.3623599278
Epoch :  0 Step :  758 loss :  47.3403180271
Epoch :  0 Step :  759 loss :  48.8748370707
Epoch :  0 Step :  760 loss :  52.1151011448
Epoch :  0 Step :  761 loss :  44.166729092
Epoch :  0 Step :  762 loss :  44.5518164982
Epoch :  0 S

...

Epoch :  11 Step :  2150 loss :  26.9929320021
Epoch :  11 Step :  2151 loss :  25.0611849938
Epoch :  11 Step :  2152 loss :  24.9045493847
Epoch :  11 Step :  2153 loss :  28.5008775244
Epoch :  11 Step :  2154 loss :  32.4856048591
Epoch :  11 Step :  2155 loss :  35.6921608986
Epoch :  11 Step :  2156 loss :  31.7934229269
Epoch :  11 Step :  2157 loss :  26.4359403391
Epoch :  11 Step :  2158 loss :  30.7922293642
Epoch :  11 Step :  2159 loss :  27.4800364429
Epoch :  11 Step :  2160 loss :  32.6625662399
Epoch :  11 Step :  2161 loss :  27.9000186823
Epoch :  11 Step :  2162 loss :  23.0935469432
Epoch :  11 Step :  2163 loss :  36.6630486077
Epoch :  11 Step :  2164 loss :  33.4175806655
Epoch :  11 Step :  2165 loss :  24.5529815043
Epoch :  11 Step :  2166 loss :  29.5216848777
Epoch :  11 Step :  2167 loss :  40.0862604708
Epoch :  11 Step :  2168 loss :  34.7491615668
Epoch :  11 Step :  2169 loss :  27.0346594733
Epoch :  11 Step :  2170 loss :  30.2934006063
Epoch :  11 S

Epoch :  12 Step :  130 loss :  32.5323523645
Epoch :  12 Step :  131 loss :  24.3871599001
Epoch :  12 Step :  132 loss :  31.4717826042
Epoch :  12 Step :  133 loss :  27.4648695959
Epoch :  12 Step :  134 loss :  35.6920336747
Epoch :  12 Step :  135 loss :  31.2974833896
Epoch :  12 Step :  136 loss :  31.0242717269
Epoch :  12 Step :  137 loss :  27.3103978347
Epoch :  12 Step :  138 loss :  30.391157744
Epoch :  12 Step :  139 loss :  24.7287226411
Epoch :  12 Step :  140 loss :  26.4285904126
Epoch :  12 Step :  141 loss :  36.4924759445
Epoch :  12 Step :  142 loss :  29.7802938479
Epoch :  12 Step :  143 loss :  39.6998439079
Epoch :  12 Step :  144 loss :  32.8483421406
Epoch :  12 Step :  145 loss :  27.8117986199
Epoch :  12 Step :  146 loss :  26.3561323472
Epoch :  12 Step :  147 loss :  23.1894856876
Epoch :  12 Step :  148 loss :  23.7458004452
Epoch :  12 Step :  149 loss :  24.0456177667
Epoch :  12 Step :  150 loss :  29.3340953811
Epoch :  12 Step :  151 loss :  25.

Epoch :  12 Step :  310 loss :  26.930260236
Epoch :  12 Step :  311 loss :  28.5216001123
Epoch :  12 Step :  312 loss :  29.5811092101
Epoch :  12 Step :  313 loss :  26.2527492483
Epoch :  12 Step :  314 loss :  25.3804004004
Epoch :  12 Step :  315 loss :  42.3977415385
Epoch :  12 Step :  316 loss :  31.4870830208
Epoch :  12 Step :  317 loss :  22.5162040377
Epoch :  12 Step :  318 loss :  27.0693815209
Epoch :  12 Step :  319 loss :  39.1603470003
Epoch :  12 Step :  320 loss :  33.4257887463
Epoch :  12 Step :  321 loss :  36.6865595491
Epoch :  12 Step :  322 loss :  21.1331316317
Epoch :  12 Step :  323 loss :  23.6135887045
Epoch :  12 Step :  324 loss :  25.4714143915
Epoch :  12 Step :  325 loss :  31.638510848
Epoch :  12 Step :  326 loss :  28.328153871
Epoch :  12 Step :  327 loss :  30.3854047208
Epoch :  12 Step :  328 loss :  24.9469032847
Epoch :  12 Step :  329 loss :  33.7054935066
Epoch :  12 Step :  330 loss :  24.9197587119
Epoch :  12 Step :  331 loss :  28.82

Epoch :  12 Step :  490 loss :  29.4336628734
Epoch :  12 Step :  491 loss :  26.960207719
Epoch :  12 Step :  492 loss :  26.0643184146
Epoch :  12 Step :  493 loss :  28.024369525
Epoch :  12 Step :  494 loss :  32.3116169143
Epoch :  12 Step :  495 loss :  30.3082020203
Epoch :  12 Step :  496 loss :  26.6975697915
Epoch :  12 Step :  497 loss :  25.8504626254
Epoch :  12 Step :  498 loss :  36.3855239588
Epoch :  12 Step :  499 loss :  29.4409483005
Epoch :  12 Step :  500 loss :  26.8205710717
Epoch :  12 Step :  501 loss :  18.5719655283
Epoch :  12 Step :  502 loss :  19.99171535
Epoch :  12 Step :  503 loss :  27.4927816322
Epoch :  12 Step :  504 loss :  26.0518201075
Epoch :  12 Step :  505 loss :  37.1299541273
Epoch :  12 Step :  506 loss :  22.9132171805
Epoch :  12 Step :  507 loss :  29.8411113874
Epoch :  12 Step :  508 loss :  29.4235624855
Epoch :  12 Step :  509 loss :  36.9157978525
Epoch :  12 Step :  510 loss :  24.7746945
Epoch :  12 Step :  511 loss :  30.161333

KeyboardInterrupt: 

In [20]:
(len(validating_data)-np.count_nonzero(model.predict(validating_data)-validating_data_class))/5000.0

0.7636

In [175]:
dumpModel(model)

In [12]:
testing_data=test.iloc[:,1:785].copy().as_matrix()/255.0
#testing_data=clf.transform(testing_data)

In [13]:
test_results=model.predict(testing_data)

In [14]:
tmp=[i for i in range(10000)]

In [15]:
csv_result=np.array(zip(tmp,test_results))

In [16]:
print csv_result

[[   0    4]
 [   1    4]
 [   2    2]
 ..., 
 [9997    3]
 [9998    7]
 [9999    3]]


In [17]:
np.savetxt('./res.csv',csv_result,delimiter=',',fmt='%d')